In [145]:
import pandas as pd

#!python -m dostoevsky download fasttext-social-network-model
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
model_dost = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [146]:
pred_asp = pd.read_csv('pred_test.csv',
                        delimiter=',')

# Из таблицы formated_reviews в таблицу с аспектами.

In [156]:
train_asp = pd.read_csv('pred_test.csv',
                        delimiter=',').drop('text', axis=1)

In [157]:
train_asp.columns

Index(['id', 'sentence', 'word', 'lemma', 'start', 'end', 'aspect',
       'sentiment'],
      dtype='object')

In [159]:
train_asp.aspect.unique()

array(['O', 'Whole', 'Service', 'Food', 'Price', 'Interior'], dtype=object)

In [161]:
sentiment_list = []
sentiment_perc_list = []
for sentence in train_asp['sentence'].values:
    messages = [sentence]
    results = model_dost.predict(messages, k=2)
    sentiment_list.append(list(results[0].items())[0][0])
    sentiment_perc_list.append(list(results[0].items())[0][1])

In [162]:
train_asp['sentiment'] = sentiment_list
train_asp['sentiment_poss'] = sentiment_perc_list

In [172]:
df_aspects = pd.DataFrame(columns=['text_id', 'category', 'mention', 'start', 'end', 'sentiment', 'sentiment_percent'])

for index, row in train_asp.iterrows():
    if row['aspect'] != '0':
        df_aspects.loc[len(df_aspects)+1] = [row['id'], row['aspect'], row['word'], row['start'], row['end'], row['sentiment'], row['sentiment_poss']]

In [173]:
df_aspects['sentiment'] = df_aspects.sentiment.map({'skip':'both', 'speech': 'positive', 'neutral':'neutral','positive':'positive', 'negative':'negative'})
df_aspects

,text_id,category,mention,start,end,sentiment,sentiment_percent
1,13823,O,Зашли,0,5,negative,0.569863
2,13823,O,в,6,7,negative,0.569863
3,13823,Whole,"""",7,8,negative,0.569863
4,13823,Whole,аппетит,8,15,negative,0.569863
5,13823,Whole,"""",15,16,negative,0.569863
...,...,...,...,...,...,...,...
11822,11770,O,",",1005,1006,neutral,0.870607
11823,11770,Whole,ресторан,1007,1015,neutral,0.870607
11824,11770,O,высокого,1016,1024,neutral,0.870607
11825,11770,O,уровня,1025,1031,neutral,0.870607


In [174]:
df_aspects.text_id.nunique()

71

In [175]:
df_aspects.sentiment.unique()

array(['negative', 'neutral', 'both', 'positive'], dtype=object)

In [176]:
df_aspects.text_id.nunique()

71

In [178]:
df_aspects.head()

,text_id,category,mention,start,end,sentiment,sentiment_percent
1,13823,O,Зашли,0,5,negative,0.569863
2,13823,O,в,6,7,negative,0.569863
3,13823,Whole,"""",7,8,negative,0.569863
4,13823,Whole,аппетит,8,15,negative,0.569863
5,13823,Whole,"""",15,16,negative,0.569863


In [179]:
df_aspects.drop('sentiment_percent', axis=1).to_csv('dev_aspect_pred.txt', header=False, index=False, sep='\t')

In [180]:
dict_of_reviews = {}
for id_review in df_aspects.text_id.unique(): 
    print(id_review)
    small_dict = {}
    small_df = df_aspects.copy()[df_aspects.text_id == id_review]
    for i in small_df.category.values:
        print(i)
        neutral = small_df[(small_df.category == i) & (small_df.sentiment == 'neutral')]['sentiment_percent'].sum()
        positive = small_df[(small_df.category == i) & (small_df.sentiment == 'positive')]['sentiment_percent'].sum()
        negative = small_df[(small_df.category == i) & (small_df.sentiment == 'negative')]['sentiment_percent'].sum()
        both = small_df[(small_df.category == i) & (small_df.sentiment == 'both')]['sentiment_percent'].sum()
        loc_max = max([neutral, positive, negative, both])
        loc_dict = {'neutral': neutral, 'positive': positive, 'negative': negative, 'both': both}
        if [neutral, positive, negative, both].count(loc_max)>1:
            if loc_max == both and loc_max == positive or loc_max == neutral and loc_max == positive:
                small_dict[i] = loc_dict['positive']
            elif loc_max == both and loc_max == negative or loc_max == neutral and loc_max == negative:
                small_dict[i] = loc_dict['negative']
            elif loc_max == negative and loc_max == positive:
                small_dict[i] = loc_dict['both']
        elif [neutral, positive, negative, both].count(loc_max)==1:
            small_dict[i] = max(loc_dict, key = loc_dict.get)
    for cat in ['Whole', 'Interior', 'Service', 'Food', 'Price']:
        if cat not in small_dict:
            small_dict[cat] = 'absence'
    dict_of_reviews[id_review] = small_dict
    print(dict_of_reviews[id_review])

13823
O
O
Whole
Whole
Whole
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
Service
Service
O
O
O
Service
O
Service
Service
Service
O
Service
Service
O
O
O
O
O
O
O
Service
O
Service
Service
O
O
O
O
O
O
O
O
O
O
O
O
O
O
Whole
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
Service
O
O
O
O
O
O
O
O
O
O
O
O
O
Food
Food
O
O
O
O
O
O
O
O
O
O
O
O
O
Price
O
O
O
O
O
Whole
O
O
O
Food
O
O
Price
O
O
O
Service
O
O
O
O
O
O
O
O
Whole
O
O
O
O
O
O
{'O': 'neutral', 'Whole': 'negative', 'Service': 'both', 'Food': 'neutral', 'Price': 'positive', 'Interior': 'absence'}
1427
O
O
O
O
Whole
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
Food
O
O
O
O
O
O
O
O
O
Service
O
O
O
O
O
O
Whole
O
Whole
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
Interior
O
O
O
Interior
O
O
O
O
O
O
O
Service
O
Service
Service
Service
O
O
O
O
O
O
O
Service
O
O
O
O
Service
O
O
O
O
O
O
O
O
O
O
O
Service
O
O
O
O
Service
Service
O
O
O
O
O
O
Service
O
Price
Food
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
Service
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
Service
Service
O


In [181]:
result = []
key_order = ['Food', 'Interior', 'Price', 'Whole', 'Service']
for key, value in dict_of_reviews.items():
    for i in key_order:
        result.append(str(key) + '\t' + str(i) + '\t' + str(value[i]))  

In [182]:
len(dict_of_reviews)

71

In [183]:
with open('dev_pred_cats_formatted.txt', 'w') as f:
    f.write('\n'.join(result))